In [72]:
import os, os.path
import numpy as np
import pandas as pd
import data_structures as ds
import setup_analysis as sa
import support_functions as sf
import importlib
import time
import warnings
import matplotlib.pyplot as plt
import model_afolu as ma

importlib.reload(ds)
importlib.reload(sa)
importlib.reload(sf)
importlib.reload(ma)

/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:683: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")


<module 'model_afolu' from '/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_afolu.py'>

In [73]:

#df_project_afolu[[x for x in df_project_afolu.columns if ("area" in x) and (("lndu" in x)) and ("cropland" in x)]].head()

In [74]:
importlib.reload(sf)
importlib.reload(ma)
df_afolu_trajectories = pd.read_csv(os.path.join(sa.dir_ref, "fake_data", "fake_data_complete.csv"))

model_afolu = ma.AFOLU(sa.model_attributes);
df_project_afolu = model_afolu.project(df_afolu_trajectories)
#df_project_afolu[[x for x in df_project_afolu.columns if "fertilizer" in x] + [x for x in df_project_afolu.columns if ("pasture" in x) and ("qty_lsmm" in x)]]
#print(df_project_afolu[[x for x in df_project_afolu.columns if ("emission" in x) and ("lsmm" in x) and ("co2e_ch4" in x)]].sum(axis = 1))
#print(df_project_afolu[[x for x in df_project_afolu.columns if ("emission" in x) and ("lsmm" in x) and ("co2e_n2o" in x)]].sum(axis = 1))
# about 2 billion kg/
#qty_lsmm_n_to_pastures_
df_project_afolu[[x for x in df_project_afolu.columns if ("emission_co2e" in x) and ("lvst" in x)]]
df_project_afolu[[x for x in df_project_afolu.columns if ("emission_co2e" in x) and ("agrc" in x)]].sum(axis = 1)
df_project_afolu[[x for x in df_project_afolu.columns if ("emission" in x) and ("frst" in x)]].head()


TEMPORARY WORK -- ASSUMED 50% DRY/WET--TO GET RUNNING - FIX TO READ FROM VARS!


/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_afolu.py:426: RuntimeWarning: divide by zero encountered in true_divide
  vec_lvst_cc_init = vec_lvst_pop_init/(vec_initial_area[ind_pstr]*vec_lvst_pstr_weights)
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_afolu.py:463: RuntimeWarning: invalid value encountered in multiply
  vec_lvst_prod_proj = vec_lvst_cc_proj*area_pstr_proj*vec_lvst_pstr_weights
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:1505: UserWarning: Missing fields 'area_agrc_crops_bevs_and_spices', 'area_agrc_crops_cereals', 'area_agrc_crops_fibers', 'area_agrc_crops_fruits', 'area_agrc_crops_herbs_and_other_perennial_crops', 'area_agrc_crops_nuts', 'area_agrc_crops_other_annual', 'area_agrc_crops_other_woody_perennial', 'area_agrc_crops_pulses', 'area_agrc_crops_rice', 'area_agrc_crops_sugar_cane', 'area_agrc_crops_tubers', 'area_agrc_crops_vegetables_and_vines', 'area_lndu_conve

,emission_co2e_co2_frst_sequestration_mangroves,emission_co2e_co2_frst_sequestration_primary,emission_co2e_co2_frst_sequestration_secondary,emission_co2e_ch4_frst_methane_mangroves,emission_co2e_ch4_frst_methane_primary,emission_co2e_ch4_frst_methane_secondary,emission_co2e_subsector_total_frst
0,-0.098780,-5.237938,-1.105878,0.001312,0.060891,0.025712,-6.354682
1,-0.098629,-5.223194,-1.093686,0.001310,0.060720,0.025428,-6.328050
2,-0.098375,-5.208907,-1.078163,0.001307,0.060554,0.025067,-6.298517
3,-0.098081,-5.195096,-1.061796,0.001303,0.060393,0.024687,-6.268590
4,-0.097764,-5.181750,-1.045307,0.001299,0.060238,0.024303,-6.238982


In [58]:
df_afolu_trajectories = pd.read_csv(os.path.join(sa.dir_ref, "fake_data", "fake_data_complete.csv"))
df_afolu_trajectories.shape                      
#self.model_attributes.get_standard_variables(df_afolu_trajectories, self.modvar_lvst_animal_weight, True, "array_base", expand_to_all_cats = True)                        

(36, 1510)

In [59]:
#vars_afolu = sa.model_attributes.build_varlist("Soil Management", variable_type = "input")
vars_afolu = sa.model_attributes.build_varlist("Agriculture", variable_type = "input")
#vars_afolu += sa.model_attributes.build_varlist("Forest", variable_type = "input")
#vars_afolu += sa.model_attributes.build_varlist("Land Use", variable_type = "input")
vars_afolu = [x for x in vars_afolu if (x not in df_afolu_trajectories.columns)]
vars_afolu.sort()

for k in vars_afolu:
    print(k)
    

ef_agrc_biomassdom_bevs_and_spices_co2
ef_agrc_biomassdom_cereals_co2
ef_agrc_biomassdom_fibers_co2
ef_agrc_biomassdom_fruits_co2
ef_agrc_biomassdom_herbs_and_other_perennial_crops_co2
ef_agrc_biomassdom_nuts_co2
ef_agrc_biomassdom_other_annual_co2
ef_agrc_biomassdom_other_woody_perennial_co2
ef_agrc_biomassdom_pulses_co2
ef_agrc_biomassdom_rice_co2
ef_agrc_biomassdom_sugar_cane_co2
ef_agrc_biomassdom_tubers_co2
ef_agrc_biomassdom_vegetables_and_vines_co2


In [407]:
importlib.reload(sf)

importlib.reload(ma)
model_afolu = ma.AFOLU(sa.model_attributes);


df_project_afolu = model_afolu.project(df_afolu_trajectories)
#df_project_afolu[[x for x in df_project_afolu.columns if "fertilizer" in x] + [x for x in df_project_afolu.columns if ("pasture" in x) and ("qty_lsmm" in x)]]
df_project_afolu[[x for x in df_project_afolu.columns if ("emission" in x) and ("lsmm" in x)]].sum(axis = 1)


/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_afolu.py:408: RuntimeWarning: divide by zero encountered in true_divide
  vec_lvst_cc_init = vec_lvst_pop_init/(vec_initial_area[ind_pstr]*vec_lvst_pstr_weights)
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_afolu.py:445: RuntimeWarning: invalid value encountered in multiply
  vec_lvst_prod_proj = vec_lvst_cc_proj*area_pstr_proj*vec_lvst_pstr_weights


ValueError: Invalid model variable ':math:\text{N}_2\text{O} Emissions from Manure Management' found in get_variable_characteristic.

In [409]:
print(model_afolu.modvar_lsmm_emissions_n2o)

:math:\text{N}_2\text{O} Emissions from Manure Management


In [80]:

#df_project_afolu = model_afolu.project(df_afolu_trajectories)
#df_cs_integrated = sf.merge_output_df_list([df_cs_integrated, df_project_afolu], sa.model_attributes, "concatenate")





In [16]:
dict_arrs_lsmm_frac_manure = sa.model_attributes.get_multivariables_with_bounded_sum_by_category(
    df_afolu_trajectories,
    model_afolu.modvar_lvst_list_mm_fractions,
    1,
    force_sum_equality = True,
    msg_append = "Energy fractions by category do not sum to 1. See definition of dict_arrs_inen_frac_energy."
)

In [26]:
sa.model_attributes.get_standard_variables(df_afolu_trajectories, model_afolu.modvar_lvst_genfactor_volatile_solids)

,genfactor_lvst_daily_volatile_solids_buffalo,genfactor_lvst_daily_volatile_solids_cattle_dairy,genfactor_lvst_daily_volatile_solids_cattle_nondairy,genfactor_lvst_daily_volatile_solids_chickens,genfactor_lvst_daily_volatile_solids_goats,genfactor_lvst_daily_volatile_solids_horses,genfactor_lvst_daily_volatile_solids_mules,genfactor_lvst_daily_volatile_solids_pigs,genfactor_lvst_daily_volatile_solids_sheep
0,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
1,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
2,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
3,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
4,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
5,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
6,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
7,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
8,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
9,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
